# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv('output_data/cities.csv')
df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)

In [6]:
Coordinate = df[["Lat", "Lng"]]
Humidity = df["Humidity"]
fig = gmaps.figure()
HeatMap = gmaps.heatmap_layer (Coordinate, weights=Humidity)
fig.add_layer(HeatMap)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
Narrow_df = df.loc[(df["Max Temp"]<=64.97) & (df["Max Temp"]>=4.92) & (df["Wind Speed"]<=8.67) & (df["Cloudiness"]==1)].dropna()
Narrow_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
50,50,kodiak,1,US,1558378763,63,39.95,-94.76,53.01,8.05
149,149,ganzhou,1,CN,1558378783,66,25.86,114.93,64.32,3.42
166,166,brainerd,1,US,1558378527,40,46.36,-94.20,63.00,5.82
266,266,alice springs,1,AU,1558378836,93,-23.70,133.88,57.20,3.36
283,283,christchurch,1,NZ,1558378744,100,-43.53,172.64,51.01,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
Hotel_df = Narrow_df[["City", "Country", "Lat", "Lng"]].copy()
Hotel_df["Hotel Name"] = ""
Hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
50,kodiak,US,39.95,-94.76,
149,ganzhou,CN,25.86,114.93,
166,brainerd,US,46.36,-94.20,
266,alice springs,AU,-23.70,133.88,
283,christchurch,NZ,-43.53,172.64,


In [68]:
params = {
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

for index, row in Hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    Name_address = requests.get(base_url, params=params)
    Name_address = Name_address.json()

    try:
        Hotel_df.loc[index, "Hotel Name"] = Name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result: No result")
        
Hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
50,kodiak,US,39.95,-94.76,Saint Joseph
149,ganzhou,CN,25.86,114.93,Ji'an
166,brainerd,US,46.36,-94.20,Brainerd
266,alice springs,AU,-23.70,133.88,Alice Springs
283,christchurch,NZ,-43.53,172.64,Christchurch


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>Hotel_df.["Hotel Name"]</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Narrow_df.iterrows()]
locations = Hotel_df[["Lat", "Lng"]]

In [73]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…